## Hodge-Compositional Edge Gaussian Process

In [1]:
import os

# set the current working directory
curr_path = os.getcwd().split("/")[:-2]
curr_path = "/".join(curr_path)
os.chdir(curr_path)

In [3]:
import numpy as np
# read dataset
from pytspl import load_dataset

sc, _, flow = load_dataset("forex")

# get the y from flow dict
y = np.fromiter(flow.values(), dtype=float)

Generating coordinates using spring layout.
Num. of nodes: 25
Num. of edges: 210
Num. of triangles: 770
Shape: (25, 210, 770)
Max Dimension: 2
Coordinates: 25
Flow: 210


In [4]:
from pytspl.hogde_gp import HodgeGPTrainer

# create the trainer object
hogde_gp = HodgeGPTrainer(sc = sc, y = y)

In [5]:
# set the training ratio
train_ratio = 0.2
# set the data normalization
data_normalization = False

# get the eigenpairs
eigenpairs = hogde_gp.get_eigenpairs()

# split the data into training and testing sets
x_train, y_train, x_test, y_test, x, y  = hogde_gp.train_test_split(train_ratio=train_ratio, data_normalization=data_normalization)

x_train: (42,)
x_test: (168,)
y_train: (42,)
y_test: (168,)


In [6]:
x_train[0], y_train[0]

(tensor(32.), tensor(4.5767))

In [10]:
from pytspl.hogde_gp.kernel_serializer import KernelSerializer

# set the kernel parameters

kernel_type = "matern" # kernel type
data_name = "forex" # data set name

# serialize the kernel
kernel = KernelSerializer().serialize(eigenpairs=eigenpairs, kernel_type=kernel_type, data_name=data_name)

In [15]:
import gpytorch
from pytspl.hogde_gp import ExactGPModel

likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(x_train, y_train, likelihood, kernel, mean_function=None)

In [16]:
type(kernel)

pytspl.hogde_gp.forex.matern.MaternKernelForex

In [17]:
import torch
output_device = "cpu"

if torch.cuda.is_available():
   model = model.to(output_device)
   likelihood = likelihood.to(output_device)

In [19]:
for param_name, param in model.named_parameters():
    print(f'Parameter name: {param_name:50} value = {param.item()}')

Parameter name: likelihood.noise_covar.raw_noise                   value = 0.0
Parameter name: mean_module.raw_constant                           value = 0.0
Parameter name: covar_module.raw_outputscale                       value = 0.0
Parameter name: covar_module.base_kernel.raw_kappa_down            value = 0.0
Parameter name: covar_module.base_kernel.raw_kappa_up              value = 0.0
Parameter name: covar_module.base_kernel.raw_kappa                 value = 0.0
Parameter name: covar_module.base_kernel.raw_mu                    value = 0.0
Parameter name: covar_module.base_kernel.raw_mu_down               value = 0.0
Parameter name: covar_module.base_kernel.raw_mu_up                 value = 0.0
Parameter name: covar_module.base_kernel.raw_h                     value = 0.0
Parameter name: covar_module.base_kernel.raw_h_down                value = 0.0
Parameter name: covar_module.base_kernel.raw_h_up                  value = 0.0


In [12]:
# train the model
model.train()
likelihood.train()
hogde_gp.train(model, likelihood, x_train, y_train)

Iteration 1/1000 - Loss: 5.387 
Iteration 2/1000 - Loss: 4.940 
Iteration 3/1000 - Loss: 4.554 
Iteration 4/1000 - Loss: 4.221 
Iteration 5/1000 - Loss: 3.936 
Iteration 6/1000 - Loss: 3.691 
Iteration 7/1000 - Loss: 3.482 
Iteration 8/1000 - Loss: 3.303 
Iteration 9/1000 - Loss: 3.150 
Iteration 10/1000 - Loss: 3.018 
Iteration 11/1000 - Loss: 2.906 
Iteration 12/1000 - Loss: 2.811 
Iteration 13/1000 - Loss: 2.729 
Iteration 14/1000 - Loss: 2.660 
Iteration 15/1000 - Loss: 2.602 
Iteration 16/1000 - Loss: 2.553 
Iteration 17/1000 - Loss: 2.512 
Iteration 18/1000 - Loss: 2.478 
Iteration 19/1000 - Loss: 2.451 
Iteration 20/1000 - Loss: 2.428 
Iteration 21/1000 - Loss: 2.409 
Iteration 22/1000 - Loss: 2.394 
Iteration 23/1000 - Loss: 2.382 
Iteration 24/1000 - Loss: 2.373 
Iteration 25/1000 - Loss: 2.365 
Iteration 26/1000 - Loss: 2.359 
Iteration 27/1000 - Loss: 2.354 
Iteration 28/1000 - Loss: 2.350 
Iteration 29/1000 - Loss: 2.346 
Iteration 30/1000 - Loss: 2.344 
Iteration 31/1000 -

In [14]:
# evaluate the model
hogde_gp.predict(model, likelihood, x_test, y_test)

Test MAE: 5.07415461470373e-05
Test MSE: 4.291597299754812e-09
Test R2: 1.0
Test MLSS: -3.288797616958618
Test NLPD: -3.5441062450408936


MultivariateNormal(loc: torch.Size([168]))